## 1. Clone repository and navigate to the directory

In [1]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model
%cd first-order-model
!pip install ffmpeg-python
!pip install imageio
!pip install imageio-ffmpeg

fatal: destination path 'first-order-model' already exists and is not an empty directory.
/content/first-order-model
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
print(os.getcwd())

/content/first-order-model


## 2. Download sample file and checkpoints
Download the folder from here: [Click Me](https://drive.google.com/drive/folders/11CuJXbxNROci7OgtUNtEvrWxTzL5mLog?usp=sharing)
and upload it in your drive

## 3. Mount your Google drive folder on Colab

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## 4. Making a display function

In [3]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))
 
    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])
 
    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani

## 5. Create a model and load checkpoints

In [ ]:
# Might need to downgrade scikit-image just in case 
# There is an issue importing demo.py file from 
!pip install scikit-image==0.17.2

In [7]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/MyDrive/deep_learning_projects/deep_fake_projects/example_1/First Order Model/vox-cpk.pth.tar')

## 6. Choose image and driver path
If you want to make your **own driver video** then you should crop the video focusing on the face.</br>
You can crop the video online from [here](https://ezgif.com/crop-video)

In [8]:
#@title Set path for the video and image in your drive
face_path = "/content/gdrive/MyDrive/deep_learning_projects/deep_fake_projects/example_1/First Order Model/Emily.png" #@param {type:"string"}
driver_path = "/content/gdrive/MyDrive/deep_learning_projects/deep_fake_projects/example_1/First Order Model/driver.mp4" #@param {type:"string"}

## 7. Start predictions

In [11]:
from demo import make_animation
from skimage import img_as_ubyte
 
source_image = imageio.imread(face_path)
reader = imageio.get_reader(driver_path)
 
fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()
 
#Resize image and video to 256x256
 
source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]
 
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True,
                             adapt_movement_scale=True)
#save resulting video
imageio.mimsave('../output.mp4', [img_as_ubyte(frame) for frame in predictions])
 
HTML(display(source_image, driving_video, predictions).to_html5_video())

  0%|          | 0/852 [00:00<?, ?it/s]

## 8. Download the resulted video

In [12]:
from google.colab import files
files.download("/content/output.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!echo "# deep_fake_projects" >> README.md
!git init
!git add README.md
!git commit -m "first commit"
!git branch -M main
!git remote add origin https://github.com/nsahsimon/deep_fake_projects.git
!git push -u origin main